### HepVu data analysis

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
#import xlsx file as a dataframe
HepVu_df = pd.read_excel('../data/HepVu_County_Opioid_Indicators_05DEC22.xlsx')

In [3]:
HepVu_df.head()

,GEO ID,State Abbreviation,County Name,Opioid Prescription Rate 2020,Narcotic Overdose Mortality Rate 2014,Narcotic Overdose Mortality Rate 2015,Narcotic Overdose Mortality Rate 2016,Narcotic Overdose Mortality Rate 2017,Narcotic Overdose Mortality Rate 2018,Narcotic Overdose Mortality Rate 2019,Narcotic Overdose Mortality Rate 2020
0,1001,AL,Autauga County,98.3,8.2,8.8,10.7,9.8,10.9,9.2,11.6
1,1003,AL,Baldwin County,65.0,18.0,20.0,16.6,15.1,14.9,14.5,27.4
2,1005,AL,Barbour County,22.8,4.4,4.5,5.7,5.8,5.2,5.7,7.6
3,1007,AL,Bibb County,24.8,17.2,16.6,22.6,21.7,23.1,19.4,27.3
4,1009,AL,Blount County,22.8,18.6,18.9,22.7,27.0,19.9,20.3,24.2


In [13]:
HepVu_df.shape

(3221, 11)

#### Updating SVI data with HepVu overdose rates

###### GEOID column needs to change: insert "0" infront of 4 character values to match other dataframe

In [4]:
HepVu_df["GEO ID"] = HepVu_df["GEO ID"].astype(str)

In [5]:
# Check the length of each value in the 'FIPS' column, and if it's 4 characters, prepend '0'
HepVu_df['GEO ID'] = HepVu_df['GEO ID'].apply(lambda x: x.zfill(5) if len(x) == 4 else x)

In [6]:
HepVu_df["GEO ID"][123]

'05023'

In [7]:
# importing SVI dataset

In [21]:
SVI_2020_df = gpd.read_file('./data/SVI data/SVI2020_US_county.gdb')

In [36]:
# get the FIPS code of the COUNTY = Autauga County	
SVI_2020_df[SVI_2020_df['FIPS'] == '01001']

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE,Shape_Length,Shape_Area,geometry
0,01,Alabama,AL,01001,Autauga,01001,"Autauga County, Alabama",594.455838,55639,0,...,0.2,0.0,0.1,2.0,0.7,0.2,0.3,2.052888,0.150259,"MULTIPOLYGON (((-86.92120 32.65754, -86.92035 ..."


In [49]:
# SVI 2018 dataset is in csv format(Issue with downloading as geo file at the moment, so using csv file for now)
SVI_2018_df = pd.read_csv('../data/SVI_2018_US_county.csv')

When we read from the CSV files rather than geo data file the FIPS column loose first zero value for some data. Because of that we need to turn it into string and add zero infront of those data

In [50]:
SVI_2018_df["FIPS"] = SVI_2018_df["FIPS"].astype(str)

In [54]:

# Check the length of each value in the 'FIPS' column, and if it's 4 characters, prepend '0'
SVI_2018_df['FIPS'] = SVI_2018_df['FIPS'].apply(lambda x: x.zfill(5) if len(x) == 4 else x)

Adding geometry column because CSV file of SVI 2018 does not contains it

In [55]:
# add "geometry" column to SVI_2018_df by merging it with SVI_2020_df matching the 'FIPS' column
SVI_2018_df = SVI_2018_df.merge(SVI_2020_df[['FIPS', 'geometry']], on='FIPS')

In [56]:
SVI_2018_df.shape

(3141, 124)

In [64]:
HepVu_df[['GEO ID','Narcotic Overdose Mortality Rate 2018']]

,GEO ID,Narcotic Overdose Mortality Rate 2018
0,01001,10.9
1,01003,14.9
2,01005,5.2
3,01007,23.1
4,01009,19.9
...,...,...
3216,72145,-9.0
3217,72147,-9.0
3218,72149,-9.0
3219,72151,-9.0


Now we need to add narcotic death rates from HepVu to SVI dataframe

In [65]:
# add all the columns from HepVu_df to SVI_2020_df by matching the FIPS column to GEO ID column
SVI_HepVu_2018_df = SVI_2018_df.merge(HepVu_df[['GEO ID','Narcotic Overdose Mortality Rate 2018']], left_on="FIPS", right_on="GEO ID")

In [66]:
SVI_HepVu_2018_df.head(3)

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,...,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,geometry,GEO ID,Narcotic Overdose Mortality Rate 2018
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,1,-999,4160,588,10.6,1.5,32290,"MULTIPOLYGON (((-107.62555 36.53832, -107.6255...",35039,80.0
1,1,ALABAMA,AL,Autauga,01001,"Autauga County, Alabama",594.443459,55200,0,23315,...,0,0,3875,508,7.1,0.9,37301,"MULTIPOLYGON (((-86.92120 32.65754, -86.92035 ...",01001,10.9
2,1,ALABAMA,AL,Blount,01009,"Blount County, Alabama",644.830460,57645,0,24222,...,0,0,6303,732,11.0,1.3,40036,"MULTIPOLYGON (((-86.96336 33.85822, -86.95967 ...",01009,19.9


In [ ]:
#get column names to a list
column_names = SVI_2020_df.columns.tolist()
column_names

In [ ]:
# rename  'Opioid Prescription Rate 2020', and 'Narcotic Overdose Mortality Rate 2020'
SVI_2020_df.rename(columns = {'Opioid Prescription Rate 2020':'OP_Rate_20', 'Narcotic Overdose Mortality Rate 2020':'NOD_Rate_2020'}, inplace = True)

In [ ]:
# save SVI_2020_df as a shapefile
SVI_2020_df.to_file("./data/SVI 2020 with HepVu/SVI2020_US_county_with_opioid_indicators.shp")